In [4]:
%pip install gradio
%pip install openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import gradio as gr
import requests

# Azure OpenAI 설정 (여기에 실제 엔드포인트와 API 키를 넣어주세요)  
AZURE_OPENAI_ENDPOINT = "https://fimtrus-openai.openai.azure.com/"
AZURE_OPENAI_API_KEY = "310a6832c2394daf97a3f446cc86ce20"
DEPLOYMENT_NAME = "fitmrus-gpt4o"

# Azure OpenAI API 호출 함수  
def chatgpt_response(prompt, history):  
    print(prompt, history)
    headers = {  
        'Content-Type': 'application/json',  
        'api-key': AZURE_OPENAI_API_KEY  
    }  
    
    # Payload for the request

    messages = []

    #System
    messages.append(
        
        {
        "role": "system",
        "content": "너는 남해 여행 전문가야."
        },
    )

    #assistant
    if len(history) > 0:
        for text in history[0]:
            messages.append({
                "role": "assistant",
                "content": text            
            })

    #user
    messages.append({
            "role": "user",
            "content": prompt
        })

    payload = {
        "messages": messages,
        "temperature": 0,
        "top_p": 0.95,
        "max_tokens": 800,
        "azureSearchEndpoint": "https://fimtrus-ai-search2.search.windows.net",
        "azureSearchKey": "oMVRj7m3JohAluhSM4N8kPhDDQiMw6ZMDBsOcnVbi6AzSeDj6vvI",
        "azureSearchIndexName": "travel-index",
        "data_sources": [
        {
            "type": "azure_search",
            "parameters": {
                "endpoint": "https://fimtrus-ai-search2.search.windows.net",
                "index_name": "travel-index",
                "semantic_configuration": "travel-semantic",
                "query_type": "semantic",
                "fields_mapping": {},
                "in_scope": False,
                "role_information": "",
                "filter": None,
                "strictness": 3,
                "top_n_documents": 5,
                "authentication": {
                "type": "api_key",
                "key": "oMVRj7m3JohAluhSM4N8kPhDDQiMw6ZMDBsOcnVbi6AzSeDj6vvI"
                },
                "key": "oMVRj7m3JohAluhSM4N8kPhDDQiMw6ZMDBsOcnVbi6AzSeDj6vvI",
                "indexName": "travel-index"
            }
        }
        ]
    }
  
    response = requests.post(  
        f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-02-15-preview",  
        headers=headers,  
        json=payload  
    )  
    
    result = response.json()  

    if result and result['choices']:
        bot_response = result['choices'][0]['message']['content'].strip()
    else:
        return '', '', []

    # print(result)
    # print(bot_response)
    history.append((prompt, bot_response))  
    citation_list = []

    if result['choices'][0]['message']['context']:
        citations = result['choices'][0]['message']['context']['citations']
        i = 0
        for citation in citations:
            i += 1
            html = f"<details><summary>참조{i}</summary><ul>{citation['content']}</ul></details>"
            citation_list.append(html)
            # citation_list.append(gr.Textbox(citation['content']))
    return '', history, ''.join(citation_list)



def greet(name):
    name_length = len(name)
    if name_length > 5:
        return "Hello!!! " + name + " World!!!"
    else:
        return ""

def select_choice(value):
    return value, value

question_list = [
    ("남해 2박 3일", "남해 2박3일 여행을 하려고하는데 관광지 추천해줘"),
    ("남해 맛집", "남해 맛집 추천해줘"),
    ("남해 친구 3박 4일", "남해 친구랑 3박 4일 여행가기 좋은 곳은 어디 있을까?"),
]

css = """
.gradio-container { 
    background-image: url('https://4kwallpapers.com/images/walls/thumbs_3t/5630.jpg'); 
}
.image-container img {
    width: 512px !important;
    height: 512px !important;
    object-fit: cover;
}
"""

with gr.Blocks(css=css) as demo:
    
    gr.Markdown("# Fimrus's ChatGPT World!!!!")
    
    with gr.Row():
        chatbot = gr.Chatbot(scale=3)
        citation = gr.HTML(label="참조")
        
    choice = gr.Dropdown(choices=question_list, label="샘플")
    with gr.Column():
        with gr.Row():
            prompt = gr.Textbox(label="프롬프트", placeholder="프롬프트를 입력하세요", scale=3)
            send_button = gr.Button('전송', scale=1)
        
        clear_button = gr.Button('지우기')

        # output = gr.Textbox(label="출력")

    send_button.click(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot, citation])
    prompt.submit(fn=chatgpt_response, inputs=[prompt, chatbot], outputs=[prompt, chatbot, citation])
    clear_button.click(lambda: None, None, chatbot, queue=False)
    
    choice.select(fn=select_choice, inputs=[choice], outputs=[choice, prompt])
        # name.submit(fn=greet, inputs=[name], outputs=[output])
        # submit_button = gr.Button("전송")
        # submit_button.click(fn=greet, inputs=[name], outputs=[output], api_name="전송")

demo.launch()

/Users/fimtrus/.pyenv/versions/3.11.9/envs/openai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.
